In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.functions import filter, col, first, round, concat, lit, to_date, when
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
import pandas as pd
import numpy as np

reviews_df = pd.read_excel('../RetailsReviews_2023-08-11.xlsx')

spark = SparkSession.builder.appName('pyspark_test').getOrCreate()

preprocessed_df = spark.createDataFrame(reviews_df)



In [2]:
# Filtering out just to show an example of "CONTENT" with "[This review was collected as part of a promotion.]"
preprocessed_df_hd = preprocessed_df.filter(
    (col('RETAILER') == 'THE HOME DEPOT') &
    (col('CONTENT').contains('This review was collected as part of a promotion'))
)


In [3]:
# Function to display df with limited text length when setting .show(truncate=False)
def show_df_with_max_length(df, column_name, max_length=100):
    df.withColumn(column_name, col(column_name).substr(1, max_length)).show(5, truncate=False)

In [4]:
# Display the 'CONTENT' column with limited text length by appying the function just created
show_df_with_max_length(preprocessed_df_hd.select('RETAILER','REVIEWER_NAME','TITLE', 'CONTENT'), 'CONTENT', max_length=52)

+--------------+-------------+---------------------------------------+----------------------------------------------------+
|RETAILER      |REVIEWER_NAME|TITLE                                  |CONTENT                                             |
+--------------+-------------+---------------------------------------+----------------------------------------------------+
|THE HOME DEPOT|Mo87         |Nice and handy Vacuum                  |[This review was collected as part of a promotion.] |
|THE HOME DEPOT|Liz68        |On of the best an the market ...       |[This review was collected as part of a promotion.] |
|THE HOME DEPOT|HeatherRN    |Love this Vac!                         |[This review was collected as part of a promotion.] |
|THE HOME DEPOT|Jamieberg1451|It’s a good vacuum                     |[This review was collected as part of a promotion.] |
|THE HOME DEPOT|Fayt413      |It works great, when its not plugged up|[This review was collected as part of a promotion.] |
+-------

In [5]:
''' 1 - Removing "[This review was collected as part of a promotion."] from Home Depot reviews
        Escaped the special characters in the regular expression pattern using backslashes,
        Added the r prefix before the pattern to indicate a raw string literal'''

preprocessed_df_hd = preprocessed_df_hd.withColumn('CONTENT', regexp_replace('CONTENT', r'\[This review was collected as part of a promotion\.\]', ''))

preprocessed_df_hd = preprocessed_df_hd.filter(
    (col('RETAILER') == 'THE HOME DEPOT')
)
show_df_with_max_length(preprocessed_df_hd.select('RETAILER','REVIEWER_NAME','TITLE', 'CONTENT'), 'CONTENT', max_length=52)

# Applying to the main df
preprocessed_df = preprocessed_df.withColumn('CONTENT', regexp_replace('CONTENT', r'\[This review was collected as part of a promotion\.\]', ''))

+--------------+-------------+---------------------------------------+------------------------------------------------------+
|RETAILER      |REVIEWER_NAME|TITLE                                  |CONTENT                                               |
+--------------+-------------+---------------------------------------+------------------------------------------------------+
|THE HOME DEPOT|Mo87         |Nice and handy Vacuum                  | I bought this vacuum 2 weeks ago. In summary:\r\n1. E|
|THE HOME DEPOT|Liz68        |On of the best an the market ...       | I intended to buy this LG CORDZERO ,because I have   |
|THE HOME DEPOT|HeatherRN    |Love this Vac!                         | I recently recieved this vacuum in the mail. It is   |
|THE HOME DEPOT|Jamieberg1451|It’s a good vacuum                     | I wanted to love this. I’d been so excited for it t  |
|THE HOME DEPOT|Fayt413      |It works great, when its not plugged up| It's easy to charge and add attachments. I don'

In [6]:
# Just to show the number of dropped rows later
original_count = preprocessed_df.count()

# 2 - Dropping duplicate rows with the same 'REVIEWER_NAME', 'TITLE', 'CONTENT' values
preprocessed_df = preprocessed_df.dropDuplicates(subset=['REVIEWER_NAME', 'TITLE', 'CONTENT'])

new_count = preprocessed_df.count()
dropped_count = original_count - new_count

# Show the counts
display(f"Original count: {original_count}")
display(f"New count after dropping duplicates: {new_count}")
display(f"Number of dropped duplicates: {dropped_count}")


'Original count: 359'

'New count after dropping duplicates: 359'

'Number of dropped duplicates: 0'

In [7]:
# 3 - Combine title and content columns
preprocessed_df = preprocessed_df.withColumn('REVIEW', concat(col('TITLE'), lit(' '), col('CONTENT')))
preprocessed_df.show(10)

+--------------+-----------+------+--------------------+---------------+--------------------+--------------------+--------------------+
|      RETAILER|    PRODUCT|RATING|           POST_DATE|  REVIEWER_NAME|               TITLE|             CONTENT|              REVIEW|
+--------------+-----------+------+--------------------+---------------+--------------------+--------------------+--------------------+
|      BEST BUY|LG A939KBGS|     5|May 26, 2022 10:3...|          2leo2|Sweet, Light, Qui...|The LG vacuum is ...|Sweet, Light, Qui...|
|      BEST BUY|LG A939KBGS|     5|Dec 16, 2022 12:4...|      88XOHOX88|          Love this!|Auto-empty leaves...|Love this! Auto-e...|
|THE HOME DEPOT|LG A939KBGS|     4|         Apr 1, 2023|     AA23250706|     It really sucks| We’ve had this v...|It really sucks  ...|
|THE HOME DEPOT|LG A939KBGS|     5|                 NaN|         AMANDA|                 NaN|                 NaN|             NaN NaN|
|THE HOME DEPOT|LG A939KBGS|     5|             

In [8]:
# 4 - Drop rows with empty 'REVIEW' column
preprocessed_df = preprocessed_df.filter(col('REVIEW') != 'NaN NaN')
preprocessed_df.show(10)

display(f"Previous Row Count: {new_count}")
new_count = preprocessed_df.count()
display(f"New Row Count: {new_count}")

+--------------+-----------+------+--------------------+---------------+--------------------+--------------------+--------------------+
|      RETAILER|    PRODUCT|RATING|           POST_DATE|  REVIEWER_NAME|               TITLE|             CONTENT|              REVIEW|
+--------------+-----------+------+--------------------+---------------+--------------------+--------------------+--------------------+
|      BEST BUY|LG A939KBGS|     5|May 26, 2022 10:3...|          2leo2|Sweet, Light, Qui...|The LG vacuum is ...|Sweet, Light, Qui...|
|      BEST BUY|LG A939KBGS|     5|Dec 16, 2022 12:4...|      88XOHOX88|          Love this!|Auto-empty leaves...|Love this! Auto-e...|
|THE HOME DEPOT|LG A939KBGS|     4|         Apr 1, 2023|     AA23250706|     It really sucks| We’ve had this v...|It really sucks  ...|
|      BEST BUY|LG A939KBGS|     5|Jun 18, 2023 4:37 PM|         AaKomo|Worth the High Cost!|We love our LG an...|Worth the High Co...|
|      BEST BUY|LG A939KBGS|     4|Jan 23, 2023 

'Previous Row Count: 359'

'New Row Count: 353'

In [9]:
''' 5 - 'POST_DATE' column contains different formats from each retailer 
         Converting them to 'yyyy-MM-dd' format'''

# Identify distinct date formats
distinct_retailer_date = preprocessed_df.select('RETAILER', 'POST_DATE').distinct()
distinct_retailer_date.show()

+--------------+--------------------+
|      RETAILER|           POST_DATE|
+--------------+--------------------+
|        AMAZON|       July 13, 2023|
|        AMAZON|        July 3, 2023|
|      BEST BUY|May 27, 2022 9:09 PM|
|        AMAZON|         May 8, 2022|
|THE HOME DEPOT|         Jun 7, 2023|
|      BEST BUY|May 25, 2022 10:0...|
|      BEST BUY|Jan 23, 2023 11:0...|
|THE HOME DEPOT|        Aug 31, 2022|
|THE HOME DEPOT|         Sep 4, 2022|
|        AMAZON|        June 3, 2023|
|THE HOME DEPOT|        Nov 26, 2021|
|THE HOME DEPOT|        Dec 18, 2022|
|      BEST BUY|Jun 3, 2022 11:19 PM|
|      BEST BUY|May 24, 2022 3:58 PM|
|        AMAZON|        June 7, 2023|
|        AMAZON|    October 22, 2022|
|        AMAZON|       June 21, 2023|
|THE HOME DEPOT|         Apr 8, 2022|
|THE HOME DEPOT|        Jun 28, 2022|
|      BEST BUY|Jul 15, 2022 10:1...|
+--------------+--------------------+
only showing top 20 rows



In [10]:
# POST_DATE formats
# Amazon: MMM dd, yyyy 
# Best Buy: MMM dd, yyyy hh:mm a
# The Home Depot: MM dd, yyyy

from pyspark.sql.functions import to_date, when

# Set the legacy time parser policy
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# Assuming 'POST_DATE' is a string column containing date information
preprocessed_df = preprocessed_df.withColumn(
    "POST_DATE",
    when(to_date('POST_DATE', 'MMM dd, yyyy hh:mm a').isNotNull(),
         to_date('POST_DATE', 'MMM dd, yyyy hh:mm a'))
    .when(to_date('POST_DATE', 'MMM dd, yyyy').isNotNull(),
         to_date('POST_DATE', 'MMM dd, yyyy'))
    .otherwise(None)  # If no format matches, set the column to None
)

# Show the DataFrame with the new date column
preprocessed_df.show()

+--------------+-----------+------+----------+---------------+--------------------+--------------------+--------------------+
|      RETAILER|    PRODUCT|RATING| POST_DATE|  REVIEWER_NAME|               TITLE|             CONTENT|              REVIEW|
+--------------+-----------+------+----------+---------------+--------------------+--------------------+--------------------+
|      BEST BUY|LG A939KBGS|     5|2022-05-26|          2leo2|Sweet, Light, Qui...|The LG vacuum is ...|Sweet, Light, Qui...|
|      BEST BUY|LG A939KBGS|     5|2022-12-16|      88XOHOX88|          Love this!|Auto-empty leaves...|Love this! Auto-e...|
|THE HOME DEPOT|LG A939KBGS|     4|2023-04-01|     AA23250706|     It really sucks| We’ve had this v...|It really sucks  ...|
|      BEST BUY|LG A939KBGS|     5|2023-06-18|         AaKomo|Worth the High Cost!|We love our LG an...|Worth the High Co...|
|      BEST BUY|LG A939KBGS|     4|2023-01-23|          Aaron|       Great product|Its great for wha...|Great product 